<a href="https://colab.research.google.com/github/HyeonhoonLee/TSA_pandas/blob/master/06_Timezone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Timezone
- 이 부분은 나중에 필요할 때 듣고 사용해도 된다.
* https://www.youtube.com/watch?v=9QTIq4Ul5AE&list=PL9mhQYIlKEhd60Qq4r2yC7xYKIhs97FfC&index=3

In [5]:
from datetime import datetime, timezone, timedelta
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates

Python 내장 패키지를 사용하면 utc timezone 을 설정할 수 있습니다.

In [6]:
timezone.utc

datetime.timezone.utc

In [7]:
# timezone unawared datetime
datetime(2019,1,1)

datetime.datetime(2019, 1, 1, 0, 0)

In [8]:
# timezone awared datetime
datetime(2019,1,1,tzinfo=timezone.utc) 

datetime.datetime(2019, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)

그러나 utc 외 timezone 을 표시하기 어렵기 때문에 pytz 패키지를 사용합니다.

In [10]:
datetime(2019,1,1,tzinfo=timezone.utc) == pytz.UTC.localize(datetime(2019,1,1))

True

In [11]:
# 한국 표준시
KST = pytz.timezone('Asia/Seoul')

In [12]:
datetime(2019,1,1,tzinfo=timezone.utc) == KST.localize(datetime(2019,1,1))

False

In [13]:
# timezone unawared datetime
mydate = datetime(2019,1,1)

In [14]:
mydate

datetime.datetime(2019, 1, 1, 0, 0)

In [15]:
# timezone awared datetime
mydate_kst = KST.localize(mydate)

In [16]:
mydate_kst

datetime.datetime(2019, 1, 1, 0, 0, tzinfo=<DstTzInfo 'Asia/Seoul' KST+9:00:00 STD>)

In [17]:
# timezone conversion
mydate_kst.astimezone(pytz.utc)

datetime.datetime(2018, 12, 31, 15, 0, tzinfo=<UTC>)

In [ ]:
# 겉으로 보이는 Date표기는 다르지만, tzinfo에 따라 timezone이 다르므로 실제 시간은 같다.(True)
mydate_kst.astimezone(pytz.utc) == mydate_kst

True

In [18]:
# replace 함수로 바꿀 경우, Y, M, D는 정확한데, timezone은 정확하지 않게 바뀐다.
mydate.replace(year=2018)

datetime.datetime(2018, 1, 1, 0, 0)

In [19]:
mydate.replace(tzinfo=KST)
# 따라서 localiz 함수를 이용하여 return하는게 정확하다.

datetime.datetime(2019, 1, 1, 0, 0, tzinfo=<DstTzInfo 'Asia/Seoul' LMT+8:28:00 STD>)

### Timezone formatting
timezone awared datetime 을 문자열로 표시할 때도 timezone 정보를 포함시킬 수 있습니다.

In [20]:
# strftime.org 참조
mydate_kst.strftime("%Y-%m-%d %H:%M:%S%Z")

'2019-01-01 00:00:00KST'

In [21]:
mydate_kst.astimezone(pytz.utc).strftime("%Y-%m-%d %H:%M:%S%Z")

'2018-12-31 15:00:00UTC'

### Current datetime

In [22]:
today = datetime.today()

**today** 메서드는 시스템에 설정된 환경정보에 따라 timezone 정보 없이 데이터를 리턴하므로 다른 환경의 다른 컴퓨터에서 같은 코드를 실행할 시 문제를 발생시킬 수 있습니다. 절대적인 값으로 현재 시각을 가져올 방법이 필요합니다. **utcnow** 함수는 UTC 기준의 현재 시각을 리턴합니다.

In [23]:
# utc 표준시 이지만 time unawared datetime 입니다.
datetime.utcnow()

datetime.datetime(2020, 9, 2, 14, 6, 55, 612983)

In [24]:
# time awared datetime 으로 만들어 사용
datetime.utcnow().astimezone(KST)

datetime.datetime(2020, 9, 2, 23, 6, 58, 164168, tzinfo=<DstTzInfo 'Asia/Seoul' KST+9:00:00 STD>)

### 최근 금요일 날짜 리턴하는 함수 만들기

요일 가져오기

In [25]:
# 요일 가져오기; 0:월, 1:화, ... , 6:일
datetime(2019,1,23).weekday()

2

In [26]:
today.weekday()

2

시간차 정보 다루기
- timedelta 를 import 해야한다.

In [27]:
# UTC 시간과 KST 시간 차이는 9시간
pytz.utc.localize(mydate) - KST.localize(mydate)

datetime.timedelta(0, 32400)

In [28]:
# UTC 시간에 9시간을 더하기
pytz.utc.localize(mydate) + timedelta(seconds=32400)

datetime.datetime(2019, 1, 1, 9, 0, tzinfo=<UTC>)

In [29]:
# 최근 마지막 금요일이 언제인지 함수 만들어보기
def last_friday():
    today = KST.localize(datetime.today())
    offset = 4 - today.weekday() % 4
    aweekago = today - timedelta(weeks=1)
    #aweekago = today.replace(day=today.day-7)  ==> 이렇게하면 첫째주에서 이전 달로 넘어가지 못한다.
    last_friday = aweekago + timedelta(days=offset)
    #last_friday = aweekago.replace(day=aweekago.day+offset)   ==> 이렇게하면 첫째주에서 이전 달로 넘어가지 못한다.
    return last_friday

In [30]:
last_friday()

datetime.datetime(2020, 8, 28, 14, 10, 28, 422862, tzinfo=<DstTzInfo 'Asia/Seoul' KST+9:00:00 STD>)